# __Capstone Project - The Battle of Neighborhoods - Week 1__

## Business Problem

The stakeholder wants to open a new cinema as company's new business and they are looking for where we can have it opened.

Customer explains that in Audience point of view, watching movie is a part of whole afternoon or night activities. Cinema should have many restaurants and shopping places nearby. Transportation is also an important factor. Audience can walk to cinema within 5 minutes from public transport facilities such as bus stop and metro station.

He wants us to concentrate on selection of cinema location according to its nearby environment. Cinema facility and rental price is not my concern. He lists out his top 10 favorite cinemas in Hong Kong with rating.

## Data

Data is the section where you describe about data that will be used to solve the problem and the source of the data.
I need to compare 5 possible locations with current cinemas in Hong Kong. Therefore, I need to find a list of Hong Kong cinema and cinemas' geographic coordinates. 

## Applying data science to solve the problem

To solve the existing problems, the start-up intends to collect location data from Foursquare and apply data science techniques and tools. The data collected will involve comparison of locations to determine which is the best location to recommend to a customer. 


##### __Import necessary library__

In [6]:
import json
import pandas as pd

##### __Download the cinema list__

In [9]:
!wget -O hk_cinema_list.json https://hkmovie6.com/api/cinemas/lists
print(" Data is Downloaded")

--2018-09-29 18:56:35--  https://hkmovie6.com/api/cinemas/lists
Resolving hkmovie6.com (hkmovie6.com)... 104.24.110.59, 104.24.111.59, 2606:4700:30::6818:6f3b, ...
Connecting to hkmovie6.com (hkmovie6.com)|104.24.110.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘hk_cinema_list.json’

hk_cinema_list.json     [ <=>                  ]  51.74K   295KB/s   in 0.2s   

2018-09-29 18:56:36 (295 KB/s) - ‘hk_cinema_list.json’ saved [52984]

 Data is Downloaded


##### __Convert the JSON data into DataFrmae__

In [11]:
cinemas_json = None
with open('hk_cinema_list.json', 'r') as f:
    cinemas_json = json.load(f)
    
cinemas = []
for data in cinemas_json['data']:
    cinemas.append({
        'Name': data['name'],
        'Address': data['address'],
        'Latitude': data['lat'],
        'Longitude': data['lon']
    })
df_cinemas = pd.DataFrame(cinemas, columns=['Name','Address','Latitude','Longitude'])
df_cinemas.head()

,Name,Address,Latitude,Longitude
0,Emperor Cinemas - Entertainment Building,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
1,The Coronet @ Emperor Cinemas - Entertainment ...,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
2,Emperor Cinemas - Tuen Mun,"3/F, New Town Commercial Arcade, 2 Tuen Lee St...",22.390776,113.975983
3,Broadway Circuit - CYBERPORT,"Shop L1 - 3, Level 1, The Arcade, 100 Cyberpor...",22.261067,114.129825
4,Broadway Circuit - PALACE IFC,"Podium L1, IFC Mall, 8 Finance Street, Central",22.285545,114.157979


In [12]:
possible_locations = [
    { 'Location': 'L1', 'Address': 'Sau Mau Ping Shopping Centre, Sau Mau Ping'},
    { 'Location': 'L2', 'Address': 'Tuen Mun Ferry, Tuen Mun'},
    { 'Location': 'L3', 'Address': 'Un Chau Shopping Centre, Cheung Sha Wan'},
    { 'Location': 'L4', 'Address': 'Prosperity Millennia Plaza, North Point'},
    { 'Location': 'L5', 'Address': 'Tsuen Fung Centre Shopping Arcade, Tsuen Wan'},
]

##### __Install the google map api client library__

In [13]:
!pip install -U googlemaps

Requirement not upgraded as not directly required: requests<3.0,>=2.11.1 in /home/jupyterlab/conda/lib/python3.6/site-packages (from googlemaps) (2.18.4)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /home/jupyterlab/conda/lib/python3.6/site-packages (from requests<3.0,>=2.11.1->googlemaps) (3.0.4)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /home/jupyterlab/conda/lib/python3.6/site-packages (from requests<3.0,>=2.11.1->googlemaps) (2.6)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /home/jupyterlab/conda/lib/python3.6/site-packages (from requests<3.0,>=2.11.1->googlemaps) (1.22)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /home/jupyterlab/conda/lib/python3.6/site-packages (from requests<3.0,>=2.11.1->googlemaps) (2018.8.24)
  Running setup.py bdist_wheel for googlemaps ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/3c/3f/25/ce6d7722dba07e5d4a12d27ab38f3

In [15]:
boss_favorite = [
    {'Name': 'Boradway Circuit - MONGKONG', 'Rating': 4.5},
    {'Name': 'Boradway Circuit - The ONE', 'Rating': 4.5},
    {'Name': 'Grand Ocean', 'Rating': 4.3},
    {'Name': 'The Grand Cinema', 'Rating': 3.4},
    {'Name': 'AMC Pacific Place', 'Rating': 2.3},
    {'Name': 'UA IMAX @ Airport', 'Rating': 1.5},
]

df_boss_favorite = pd.DataFrame(boss_favorite, columns=['Name','Rating'])
df_boss_favorite

,Name,Rating
0,Boradway Circuit - MONGKONG,4.5
1,Boradway Circuit - The ONE,4.5
2,Grand Ocean,4.3
3,The Grand Cinema,3.4
4,AMC Pacific Place,2.3
5,UA IMAX @ Airport,1.5




With above data, I can build a content-based recommender systems to resolve the problem.

Combine with FourSquare API on counting how many different venues (Food, Transport, Night Life) and Hong Kong cinema list, a cinema nearby venues matrix can be built. Stakeholder's favorite list is the profile to combine with cinema nearby venues matrix to become a weighted matrix of favorite cinema.

The weighted matrix can be applied on 5 possible locations with venues information to generate a ranking result. The the top one on the ranking list can be recommended to the stakeholder.

# Methodology

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

The methodology I used is pretty straightforward. I will try out a number of different machine learning / statistical learning methods after splitting the data into training and test sets, and find out which ones are best at describing the best location for cinema. 

# Results

Results section where you discuss the results.

# Discussion

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results. 

# Conclusion

Conclusion section where you conclude the report. Without a lot more work in the initial data exploration and methodology phase, it would not be possible to figure out which location would be best fit for cinema.
